# Module 3 - Amazon Q cross-app Using filters with your index

### Use Amazon Q Business cross-app index to search relevant content

In this module, we will call the Amazon Q Business `search_relevant_content` API with various filters to perform a refined search. This search is useful for building more targeted Retrieval Augmented Generation (RAG) based generative results. We will review and compare the response of the API which will return textual chunks of relevant data with filter criteria applied.

<div class="alert alert-block alert-warning"> <b>IMPORTANT:</b> Please make sure that you select <b>"Python 3 (ipykernel)"</b> kernel for this notebook from the top right corner, if one is not selected already. </div>

<div class="alert alert-block alert-info"> <b>INFORMATION:</b> If you have just completed module 2 in this workshop then you can skip this step 1.<br>
If you have not completed module 2 then let's install some pre-requisites. Run the following code-block.</div>

# Step 1

In [ ]:
!python -m pip install boto3

## Step 2
For some of the API calls used in this notebook, we will need to know the following pieces of information for the Amazon Q Business application.  

- The application ID  
- The retriever ID  
- The index ID  

The following code will call Amazon Q Business APIs to fetch these values.

In [ ]:
import boto3

q_business_client = boto3.client('qbusiness')
response_app = q_business_client.list_applications()

# Fetch the Q Business App ID, Retriever ID and Index ID
Q_BIZ_APP_ID = ""
Q_RETRIEVER_ID = ""
Q_INDEX_ID = ""


for r in response_app["applications"]:
    if 'aim333-module-2' in r['displayName']:
        Q_BIZ_APP_ID=r['applicationId']

if Q_BIZ_APP_ID:
    response_ret = q_business_client.list_retrievers(applicationId=Q_BIZ_APP_ID)
    Q_RETRIEVER_ID = response_ret['retrievers'][0]['retrieverId']

if Q_RETRIEVER_ID:
    response_index = q_business_client.list_indices(applicationId=Q_BIZ_APP_ID)
    Q_INDEX_ID = response_index['indices'][0]['indexId']

print(f"Application ID is: {Q_BIZ_APP_ID}\nRetriever ID is: {Q_RETRIEVER_ID}\nIndex ID ID is: {Q_INDEX_ID}")

# Step 3
This notebook is going to walk us through on how to leverage the Search Relevant Content API using filters.
When calling the SRC API we can specify which document attributes we would like to filter on. Every document has structural attributes or meta data attached to it. Document attributes can include information such as document title, document author, time created, document type, etc.
In this notebook we are going to create a new custom document attribute called Department and update our meta JSON files associated with the sample documents by adding this attribute.
Lets first start by displaying a document sample meta file that we have in our S3 bucket.
Within our workshop test account we have a bucket called s3://amazon-q-data-source-############, which contains fictious support ticket PDF files along with their associated meta files.<br>
The following code will list some of these meta file names.



In [ ]:
import json
import sagemaker

role = sagemaker.get_execution_role()
account_id = role.split(':')[4]

bucket_name_tickets = f'amazon-q-data-source-{account_id}'
s3_client = boto3.client('s3')
# Get list of meta data file objects in bucket
paginator = s3_client.get_paginator('list_objects_v2')
files = []

for page in paginator.paginate(Bucket=bucket_name_tickets):
    if 'Contents' in page:
        for obj in page['Contents']:
            if obj['Key'].startswith('ticket') and obj['Key'].endswith('metadata.json'):
                files.append(obj['Key'])
# Display file names
print(json.dumps(files[:3], indent=4))

As you can see these meta files in S3 follow a certain naming convention <b>filename.metadata.json</b> <br>
For example ticket_0.pdf has an associated meta file called ticket_0.pdf.metdata.json in the same directory.

Lets look at the content of one of these meta JSON files.

In [ ]:
# Read and parse a JSON file from S3
response = s3_client.get_object(Bucket=bucket_name_tickets, Key=files[0])
print(json.dumps(json.loads(response['Body'].read().decode('utf-8')),indent=4))

# Step 4
The meta JSON file contains a section called Attributes.  
<div class="alert alert-block alert-info"> <b>INFORMATION:</b> We modifed the value in the reserved attribute <b>_source_url</b> to point at a public version of this document. This gets surfaced as the citation link.</div>
We are going to create a new custom attribute called Department.
The following script will enumerate over all the meta files in our S3 bucket, modifying each by adding the attribute <b>Department</b> and randomly assigning it to HR or Finance.


In [ ]:
import random
for file_key in files:
    response = s3_client.get_object(Bucket=bucket_name_tickets, Key=file_key)
    json_content = json.loads(response['Body'].read().decode('utf-8'))
    if random.randint(1,2)== 1 :
        json_content['Attributes']['Department'] = 'HR'
    else:
        json_content['Attributes']['Department'] = 'Finance'
    # Upload the updated JSON back to S3
    s3_client.put_object(Bucket=bucket_name_tickets,Key=file_key,Body=json.dumps(json_content))

# Lets print the contents again
response = s3_client.get_object(Bucket=bucket_name_tickets, Key=files[0])
print(json.dumps(json.loads(response['Body'].read().decode('utf-8')),indent=4))

print(f"{len(files)} Meta files updated")

Lets recap what we have done so far.
1. We retrieved our Q Business Application ID, Retriever ID and Index ID.
1. We viewed some meta data JSON files that are associated with our documents.
2. We enumerated over all our document meta files and added a new attribute called Department to each, and randomly set it to either HR or Finance.

Now its time to test calling the search relative content API using Department as a filter.  
Before we test, we need to run and resync the S3 data connector. Head back to our Q Business application aim333-module-2 console, select the S3DataSource, and click on sync now and observe the status. This will take a few minutes to complete. When sync is complete we can proceed to the next step.

# Step 5
#### Pre-requisites for Identity Aware API call.
 
In order to be able to call Amazon Q Business's data APIs, we will need to acquire credentials that are tagged to a specific user id (in this case, an email address). We have pre-deployed a mechanism that will help generate this credentials with a helper function. First we will need to acquire some necessary details that will help us generate the identity-aware SigV4 AWS credentials. Specifically, we will require the following details.
 
- `issuer`: the issuer URL
- `client_id`: a client_id for the OIDC client
- `client_secret`: a client_secret for the OIDC client
- `role_arn`: the IAM role to assume, this is the role that has Amazon Q Business permissions
- `region`: `us-east-1` this is the current region where our Amazon Q Business Application is setup 
- `email`: your email address (can be a fictional email address of the format user@email.com)
 
To obtain the values, execute the next code cell which will read a JSON file from an S3 bucket. 

In [ ]:
s3_client = boto3.client('s3')
role = sagemaker.get_execution_role()
account_id = role.split(':')[4]
bucket_name = f'amazon-q-tvm-{account_id}'
file_key = 'tvm_values.json'

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
creds = json.loads(response['Body'].read().decode('utf-8'))["TVMOidcIssuerStack"]
creds["IssuerUrlOutput"] = creds["IssuerUrlOutput"].rstrip('/')

Lets get our identity aware credentials

In [ ]:
from utils.tvm_client import TVMClient

email_address = "john_doe@email.com"

token_client = TVMClient(
        issuer=creds["IssuerUrlOutput"],
        client_id=creds["QbizTVMClientID"],
        client_secret=creds["QbizTVMClientSecret"],
        role_arn=creds["QBizAssumeRoleARN"],
        region="us-east-1"
)

# Get Sigv4 credentials using TVM
credentials = token_client.get_sigv4_credentials(email=email_address)
qbiz = boto3.client("qbusiness", **credentials)

## Step 6
Lets run some queries.

In [ ]:
#myQuestion = "What are the reasons for keyboard failures?"
myQuestion = "What is the remediation of password not working?"

Let's test this client by calling the `search_relevant_content` API `without filtering` first.

In [ ]:
qbiz = boto3.client("qbusiness", region_name="us-east-1", **credentials)


search_params = {'applicationId': Q_BIZ_APP_ID,
    'contentSource': {
        'retriever': {
            'retrieverId': Q_RETRIEVER_ID
            }
    },
    'queryText': myQuestion,
    'maxResults': 5
}


def call_search_relevant_content(search_params):
    try:
        search_response = qbiz.search_relevant_content(**search_params)
        return search_response['relevantContent']
    except Exception as e:
        print(e)


relevant_content = call_search_relevant_content(search_params)

In [ ]:
full_context = ""
count=0

for chunks in relevant_content:
    full_context = full_context + chunks['content'] + "\n"
    count = count + 1

print(f"{count}  Chunks of relevant content was retrieved from the Search Relevant Content API \n")
print("The full context is:- \n")
print(full_context)

In [ ]:
modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')

In [ ]:
def bedrockConverse(query, context, model):
    SYSTEM_PROMPT=""""
    You are a helpful AI assistant who answers question correctly and accurately about a AcmeCompany's IT tickets. Do not makeup answers and only answer from the provided knowledge.
    """
    messages = [{"role": "user", "content": [{"text": f"Given the full context: {context}\n\nAnswer this question accurately: {query}"}]}]
    converse_params = {
        "modelId": model,
        "messages": messages,
        "system": [{"text": SYSTEM_PROMPT}]
    }
    ai_response = bedrock_client.converse(**converse_params)
    return ai_response['output']['message']['content'][0]['text']

In [ ]:
print(bedrockConverse(myQuestion, full_context, modelId))

Now let's try this again, this time we will be calling the `search_relevant_content` API with filtering. We will use the `attributesFilter` `equalsTo` filter to search for documents with a specific attribute value of <b>HR</b>.  
Note attribute filter name is case sensitive.  
[For information on this API click here to see Boto3 SDK documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/qbusiness/client/search_relevant_content.html#:~:text=all%20supplied%20filters.-,equalsTo,-(dict)%20%E2%80%93)

In [ ]:
search_params = {
    'applicationId': Q_BIZ_APP_ID,
    'contentSource': {
        'retriever': {
            'retrieverId': Q_RETRIEVER_ID
            }
    },
    'queryText': myQuestion,
    'maxResults': 5,
    'attributeFilter': {
        'equalsTo': {
            'name': 'Department',
            'value': {
                'stringValue': 'HR'
            }
        }
    }
}

relevant_content = call_search_relevant_content(search_params)

In [ ]:
full_context = ""
count = 0
for chunks in relevant_content:
    full_context = full_context + chunks['content'] + "\n"
    count = count + 1

print(f"{count}  Chunks of relevant content was retrieved from the Search Relevant Content API \n")
print("The full context is:- \n")
print(full_context)

You can now compare the chunks retrieved with the filter and what was retrieved before applying the filter and see the difference.

In [ ]:
print(bedrockConverse(myQuestion, full_context, modelId))

As you can see the results differ from our previous request where no filtering was applied. Lets apply the department filter of Finance.

In [ ]:
search_params = {
    'applicationId': Q_BIZ_APP_ID,
    'contentSource': {
        'retriever': {
            'retrieverId': Q_RETRIEVER_ID
            }
    },
    'queryText': myQuestion,
    'maxResults': 5,
    'attributeFilter': {
        'equalsTo': {
            'name': 'Department',
            'value': {
                'stringValue': 'Finance'
            }
        }
    }
}

relevant_content = call_search_relevant_content(search_params)

In [ ]:
full_context = ""
count=0

for chunks in relevant_content:
    full_context = full_context + chunks['content'] + "\n"
    count = count + 1

print(f"{count}  Chunks of relevant content was retrieved from the Search Relevant Content API \n")
print("The full context is:- \n")
print(full_context)

In [ ]:
print(bedrockConverse(myQuestion, full_context, modelId))

## Step 7
So now that we have demonstrated how to apply filtering at the API level using document attributes, lets cover one more aspect of document attributes and how they can be made part of the index, and searchable through queries.  
To do that we need to create this custom attribute on our Q Index under metadata controls which will ultimately be populated with this document attribute value during ingestion and made searchable.  
We will do this programtically next by calling the update index API and adding the Deparment attribute as searchable. Note this can also be done via the Q Business console under the section Metadata controls.  
<div class="alert alert-block alert-info"> <b>INFORMATION:</b> Any data source can populate this Q Index custom attribute during syncing with the correct field mapping.</div>

In [ ]:
# add the new attribute 'Department' name to the Index
response = q_business_client.update_index(
    applicationId=Q_BIZ_APP_ID,
    indexId=Q_INDEX_ID,
    documentAttributeConfigurations=[
        {
            'name': 'Department',
            'type': 'STRING',
            'search': 'ENABLED'
        },
    ]
)

# Step 8
With our Q Index now containing the new custom attribute 'Department', we now need to update our S3 data connector with this custom attribute field mapping. This will enable the S3 data connector during ingestion to map the attribute in our meta JSON file to this custom Q Index attribute. <br>The following code will display the current S3 data connector settings.  
  
To add the new field mapping to our S3 data connector, we are going to head back to the console and make the change there. For guidance refer back to the workshop documentation, Module 3, Amazon Q Index With Filters, step 8.  
<div class="alert alert-block alert-info"> <b>INFORMATION:</b> Please switch to the console to modify S3 data connector.</div>

In [ ]:
# Fetch a list of data sources that our Q Business Index has
response = q_business_client.list_data_sources(
    applicationId=Q_BIZ_APP_ID,
    indexId=Q_INDEX_ID
)

# Select the specific data source. In our case we will select the S3 datasource that has been created as part of the workshop.
Q_DATASOURCE_ID = ''
for dataSource in response['dataSources']:
    if dataSource['type'] in 'S3':
        Q_DATASOURCE_ID = dataSource['dataSourceId']
        print(f"S3 Data Source found with ID {Q_DATASOURCE_ID}")

# lets take a look at our current configuration for the S3 data connector
response = q_business_client.get_data_source(
    applicationId=Q_BIZ_APP_ID,
    indexId=Q_INDEX_ID,
    dataSourceId=Q_DATASOURCE_ID
)


print(json.dumps(response['configuration'], indent=4))

<div class="alert alert-block alert-info"> <b>INFORMATION:</b> Please consult workshop documentation on adding a field mapping to a data source from the console.</div>

Now that we have made this attribute searchable, you can run a new query against the Search Relevant Content API and include the attribute name or value in your query. Refer to Step 6 in this notebook to run a query.

# Conclusion
---

In this module we learnt how to call `SearchRelevantContent` API and apply filtering. In our example we created a new custom attribute called Department and inserted that into our S3 document meta files.

The search_relevant_content API in Amazon Q Business allows users to filter search results based on various document attributes or metadata fields. Filters can be applied using logical operations such as andAllFilters, orAllFilters, and notFilter. Specific filter types include equalsTo, containsAll, containsAny, greaterThan, greaterThanOrEquals, lessThan, and lessThanOrEquals. These filters support different attribute value types like stringValue, stringListValue, longValue, and dateValue. By applying these filters, users can refine their search to return only the most relevant content items, enhancing the precision and relevance of the search results.  
[For more information on this API click here to see Boto3 SDK documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/qbusiness/client/search_relevant_content.html#:~:text=all%20supplied%20filters.-,equalsTo,-(dict)%20%E2%80%93)

This concludes our workshop! Thanks for joining us and please take a moment to fill out the Survey for Session